[FROM HERE](https://github.com/drcrane/xvfb-alpine-docker)


In [ ]:
# org Dockerfile from repo

```bash
# on Jan 26, 2019
cat <<EOF >Dockerfile_org_from_repo
FROM alpine:3.8

RUN apk update

RUN adduser -D -g users user
RUN mkdir -p /home/user \
	&& chown user:users /home/user

RUN apk add xvfb x11vnc fluxbox xdpyinfo st vim terminus-font \
	&& sed -r -i "s/\[exec\] \(xterm\) \{xterm\}/\[exec\] \(st\) \{st -f 'xos4 Terminus-14'\}/" /usr/share/fluxbox/menu \
	&& [[ ! -d /opt ]] && mkdir /opt \
	&& rm -vrf /var/cache/apk/*

COPY bootstrap.sh /opt

USER user
ENV HOME /home/user
WORKDIR /home/user
CMD ["/opt/bootstrap.sh"]
EOF

# build


In [ ]:
docker build --tag alpine-xvfb:beta1 Dockerfile_org_from_repo

# update alpine version


In [ ]:
```bash
# on Jan 26, 2019
cat <<EOF >Dockerfile
FROM alpine:3.18.5 

RUN apk update

RUN adduser -D -g users user
RUN mkdir -p /home/user \
	&& chown user:users /home/user

RUN apk add xvfb x11vnc fluxbox xdpyinfo st vim terminus-font \
	&& sed -r -i "s/\[exec\] \(xterm\) \{xterm\}/\[exec\] \(st\) \{st -f 'xos4 Terminus-14'\}/" /usr/share/fluxbox/menu \
	&& [[ ! -d /opt ]] && mkdir /opt \
	&& rm -vrf /var/cache/apk/*

COPY bootstrap.sh /opt

USER user
ENV HOME /home/user
WORKDIR /home/user
CMD ["/opt/bootstrap.sh"]
EOF

# build


In [ ]:
docker build --no-cache --tag alpine-xvfb:latest .

# run


In [ ]:
docker run --rm -it alpine-xvfb:latest

# docker container ip


In [ ]:
docker inspect -f \
'{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' <CONTAINER ID>
docker inspect -f \
'{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' ffc7f4bbc880

#

export DISPLAY=:0
xhost +
vncviewer 172.17.0.2:0


# open shell inside container


In [ ]:
docker  exec -ti  ffc7f4bbc880 /bin/ash

# with xeyes


In [ ]:
```bash
# on Dez 2023
cat <<EOF >Dockerfile_alpine_xeyes_xterm_htop
FROM alpine:3.18.5 

RUN apk update

RUN adduser -D -g users user
RUN mkdir -p /home/user \
	&& chown user:users /home/user

RUN apk add xvfb x11vnc fluxbox xdpyinfo st vim terminus-font xterm xeyes htop \
	&& sed -r -i "s/\[exec\] \(xterm\) \{xterm\}/\[exec\] \(st\) \{st -f 'xos4 Terminus-14'\}/" /usr/share/fluxbox/menu \
	&& mkdir -p /opt \
	&& rm -vrf /var/cache/apk/*

COPY bootstrap.sh /opt

USER user
ENV HOME /home/user
WORKDIR /home/user
CMD ["sh","/opt/bootstrap.sh"]
EOF

# build


In [ ]:
docker build --no-cache --tag alpine-xvfb-xeyes:latest  -f Dockerfile_alpine_xeyes "."

# run


In [ ]:
docker run --rm -it alpine-xvfb-xeyes:latest

# container id


In [ ]:
echo $(docker ps --format "{{.ID}}"  | cut -d " " -f1)

# get container ip address


In [ ]:
# docker inspect -f \
#'{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' <CONTAINER ID>
docker inspect -f \
'{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' $(docker ps --format "{{.ID}}"  | cut -d " " -f1)

# start vncviewer


export DISPLAY=:0
xhost +
vncviewer <container ip>

#

export DISPLAY=:0
xhost +
vncviewer 172.17.0.2:0

# all together

export DISPLAY=:0
xhost +
vncviewer $(docker inspect -f \
'{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' $(docker ps --format "{{.ID}}" | cut -d " " -f1))


# alpine firefox
[TIP FROM HERE THX!!](https://unix.stackexchange.com/questions/731296/alpine-not-installing-latest-package)


# install rust
# FROM HERE
# https://hub.docker.com/layers/rustlang/rust/nightly-alpine3.17/images/sha256-3ae253190c5d5bce41e4e17dc44b6fbf2c45f279be1554f676ed7ffa67baad26?context=explore

```bash
# on Dez 2023
cat <<EOF >Dockerfile_alpine_firefox_rust
FROM alpine:3.18.5

RUN echo @latest https://dl-cdn.alpinelinux.org/alpine/latest-stable/main >> /etc/apk/repositories && \
    echo @latest https://dl-cdn.alpinelinux.org/alpine/latest-stable/community >> /etc/apk/repositories && \
    echo @14.20.1 https://dl-cdn.alpinelinux.org/alpine/v3.14/main >> /etc/apk/repositories

# Install dependencies
RUN apk --no-cache --update add \
    libsrt@latest \
    chromium@latest \
    firefox@latest

RUN adduser -D -g users user

RUN mkdir -p /home/user \
	&& chown user:users /home/user

RUN apk update \
	&& apk add --no-cache --update xvfb x11vnc fluxbox xdpyinfo st vim terminus-font xterm xeyes htop sudo \
	&& sed -r -i "s/\[exec\] \(xterm\) \{xterm\}/\[exec\] \(st\) \{st -f 'xos4 Terminus-14'\}/" /usr/share/fluxbox/menu \
	&& mkdir -p /opt \
	&& rm -vrf /var/cache/apk/*

# install rust

ENV RUSTUP_HOME=/usr/local/rustup CARGO_HOME=/usr/local/cargo PATH=/usr/local/cargo/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin

RUN /bin/sh -c set -eux; \
    apkArch="$(apk --print-arch)"; \
    case "$apkArch" in x86_64) rustArch='x86_64-unknown-linux-musl' ;; \
                      aarch64) rustArch='aarch64-unknown-linux-musl' ;; \
                      *) echo >&2 "unsupported architecture: $apkArch"; exit 1 ;; \
        esac; 

COPY bootstrap.sh /opt

USER user
ENV HOME /home/user
WORKDIR /home/user
CMD ["sh","/opt/bootstrap.sh"]
# CMD ["/bin/sh -c set -eux","/opt/bootstrap.sh"]

EOF

```


# build

In [ ]:
docker build --no-cache --tag alpine-firefox:latest  -f Dockerfile_alpine_firefox "."
docker build  --tag alpine-firefox-rust:latest  -f Dockerfile_alpine_firefox_rust "."
# --progress=plain
docker build  --progress=plain --tag alpine-firefox-rust:latest  -f Dockerfile_alpine_firefox_rust "."

# run

In [ ]:
docker run --rm -it alpine-firefox:latest

# start vncviewer

In [ ]:
export DISPLAY=:0
xhost +
vncviewer $(docker inspect -f \
'{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' $(docker ps --format "{{.ID}}" | cut -d " " -f1))


# install rust

# install rust
# FROM HERE
# https://hub.docker.com/layers/rustlang/rust/nightly-alpine3.17/images/sha256-3ae253190c5d5bce41e4e17dc44b6fbf2c45f279be1554f676ed7ffa67baad26?context=explore

ENV RUSTUP_HOME=/usr/local/rustup CARGO_HOME=/usr/local/cargo PATH=/usr/local/cargo/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin

RUN /bin/sh -c set -eux; \
    apkArch="$(apk --print-arch)"; \
    case "$apkArch" in x86_64) rustArch='x86_64-unknown-linux-musl' ;; \
                      aarch64) rustArch='aarch64-unknown-linux-musl' ;; \
                      *) echo >&2 "unsupported architecture: $apkArch"; exit 1 ;; \
        esac;  && \
                            
        url="https://static.rust-lang.org/rustup/dist/${rustArch}/rustup-init";   && \
        wget "$url";   && \
        chmod +x rustup-init;   && \
        /rustup-init -y --no-modify-path --default-toolchain nightly;  && \
        rm rustup-init; && \
        chmod -R a+w $RUSTUP_HOME $CARGO_HOME;   && \
        rustup --version; && \
        cargo --version; && \
        rustc --version;

#[rust dockerfile](https://github.com/rust-lang/docker-rust/blob/c9d5ef80a065e00761652951a22064f0134614aa/1.74.0/alpine3.17/Dockerfile)